In [11]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, OrdinalEncoder
from sklearn.linear_model import LinearRegression, Lasso,ElasticNet,Ridge
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_transformer

In [12]:
# import du csv
df = pd.read_csv('data/df_clean_modelisation.csv')

# selection des features et de la cible
X = df.drop('charges', axis=1)
X=df
y = df.charges
print(df.columns)
df.head()


Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges',
       'bmi_poids_normal', 'bmi_surpoids', 'bmi_obésité',
       'bmi_obésité_morbide', 'region_northeast', 'region_northwest',
       'region_southeast', 'region_southwest'],
      dtype='object')


,age,sex,bmi,children,smoker,region,charges,bmi_poids_normal,bmi_surpoids,bmi_obésité,bmi_obésité_morbide,region_northeast,region_northwest,region_southeast,region_southwest
0,19,female,27.900,0,yes,southwest,16884.92400,False,True,False,False,False,False,False,True
1,18,male,33.770,1,no,southeast,1725.55230,False,False,True,False,False,False,True,False
2,28,male,33.000,3,no,southeast,4449.46200,False,False,True,False,False,False,True,False
3,33,male,22.705,0,no,northwest,21984.47061,True,False,False,False,False,True,False,False
4,32,male,28.880,0,no,northwest,3866.85520,False,True,False,False,False,True,False,False


### Selection features: 'bmi','age','children','sex','smoker''region'

In [18]:



# Création du train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=0)



# Séparation des colonnes numériques et catégorielles

num_col = ['age','children']



# pipeline pour les valeurs numériques
# cela permet de créer une succession d'étapes (ici il n'y en a qu'une)

my_num_pipe = make_pipeline(StandardScaler())

my_cat_pipe = make_pipeline(OneHotEncoder())

my_ordi_pipe = make_pipeline(OrdinalEncoder())
                            
# ColumnTransformer permet de faire des opérations sur des sélections de colonnes
preprocessor = make_column_transformer((my_num_pipe,num_col),
                                       (my_ordi_pipe,['sex','smoker','bmi_poids_normal', 'bmi_surpoids', 'bmi_obésité',
                                        'bmi_obésité_morbide','region_southeast']))


# affichage

tab = preprocessor.fit(df)

cols = tab.get_feature_names_out()

df_transform = pd.DataFrame(preprocessor.fit_transform(df), columns=cols)

display(df_transform.head())



from sklearn.model_selection import GridSearchCV

model= make_pipeline(preprocessor, ElasticNet())
model.fit(X_train, y_train)
model.score(X_test, y_test)

# Définir les valeurs à tester pour alpha et l1_ratio

param_grid = {'elasticnet__alpha': np.arange(0.1, 1,100),
              'elasticnet__l1_ratio': np.arange(0.1,1,0.01)}

# création du modèle

model= make_pipeline(preprocessor,PolynomialFeatures(degree=2),ElasticNet(max_iter=1000000))

# creation de l'objet GridSearch

grid_search = GridSearchCV(model,param_grid, cv=5)

# recherche des hyperparamètres et Afficher les meilleures valeurs trouvées

grid_search.fit(X_train, y_train)

print("Meilleurs paramètres : ",grid_search.best_params_)

# Utiliser le modèle avec les meilleures valeurs
best_model = grid_search.best_estimator_

best_model.score(X_test, y_test)

print("Score : ", best_model.score(X_test, y_test))

# calcul RSME

rmse= np.sqrt(mean_squared_error(y_test,best_model.predict(X_test)))
print("RSME: ",rmse)

,pipeline-1__age,pipeline-1__children,pipeline-2__sex,pipeline-2__smoker,pipeline-2__bmi_poids_normal,pipeline-2__bmi_surpoids,pipeline-2__bmi_obésité,pipeline-2__bmi_obésité_morbide,pipeline-2__region_southeast
0,-1.440418,-0.909234,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,-1.511647,-0.079442,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,-0.799350,1.580143,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,-0.443201,-0.909234,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,-0.514431,-0.909234,1.0,0.0,0.0,1.0,0.0,0.0,0.0


Meilleurs paramètres :  {'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.9899999999999995}
Score :  0.8436686963708833
RSME:  4996.382993166534
